In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
from selenium.common.exceptions import NoSuchElementException
import json
from tqdm import tnrange,tqdm_notebook
# setup Driver|Chrome : 크롬드라이버를 사용하는 driver 생성

with open('review_daegu_33689.json','r',encoding='UTF8') as fp: #클라우드 대구 리뷰데이터 기준으로 
    review = json.load(fp)

driver = webdriver.Chrome('C:\\Users\\Kim\\Desktop/chromedriver')
driver.implicitly_wait(1.5)


language = []
id_trans={}
l_id=[]
k=1

for listing_id, rev_lst in review.items():
    comments = []
    trans = []
    l_id.append(listing_id)
    print(str(k)+'번째 집 시작')
    
    for i in tnrange(len(rev_lst), desc='변역루프'):
        rev = rev_lst[i]
        comment = rev['comments']
        comments.append(comment)
        lang = rev['language']
        language.append(lang)
        if lang == 'en':
            trans.append(comment) # 영어면 번역 안하게끔 하기 위해
            print(str(i+1)+'번째는 영어입니다. 번역을 하지 않습니다.')
        else:
            url = 'https://papago.naver.com/?' +'sk='+lang+'&tk=en&st='+comment
            driver.get(url)
            time.sleep(1.5)
            try:
                target = driver.find_element_by_xpath('//*[@id="txtTarget"]/span')
                t_trans= target.text
                trans.append(t_trans)
                
            except NoSuchElementException :  
                #^^,5000자이상일시 일어나는 현상
                #정규표현식, 5000자넘으면 하는 방법 생각해내야함
                print(str(i+1)+'번째에서 오류가 났습니다.')
                trans.append('error')
    
    print(str(k)+'번째 집 완료')
    print(str(len(language))+'개의 리뷰 번역완료')
    id_trans[listing_id] = trans # 딕셔너리 (id : id별 번역된리뷰들의리스트)
    k = k+1
    
with open('daegu_id_trans.json', 'w', encoding='UTF8') as fp:  # encoding 설정 해줘야 한다.
    json.dump(id_trans, fp, ensure_ascii=False)

print('번역 끝')